In [1]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import os
import numpy as np
from astropy.time import Time
from astropy.table import Table
from astropy import units as u
import seaborn as sns
from thesis import output_folder, big_fontsize, base_width, base_height, dpi
from IPython.display import display, Markdown
import numpy as np

In [2]:
# plt.rcParams["font.family"] = "sans-serif"
plt.rc('text', usetex=True)
plt.rc('text.latex', preamble=r'\usepackage{romanbar}')

In [3]:
redshift = 1. + 0.266

In [4]:
# swift_ref_time = Time(59007.690095, format='mjd')
swift_ref_time = Time(51910, format='mjd')

In [5]:
#from avro

refmag = {
    "g": 20.031999588012695,
    "r": 19.325000762939453,
    "i": 18.906999588012695
}


In [6]:
fp = pd.read_csv("data/tywin.csv", sep=",", index_col=0)
print(fp.columns)

FileNotFoundError: [Errno 2] No such file or directory: 'data/tywin.csv'

In [ ]:
photometry = pd.read_csv("data/tywin.csv", sep=",")
mask = np.array([x in ["r", "g", "i"] for x in photometry["filter"]])
photometry = photometry[mask]
print(photometry)

In [ ]:
swift_data = pd.DataFrame()

filtermap = {
    "UUU": "U",
    "UM2": "UVM2",
    "UW2": "UVW2",
    "UW1": "UVW1"
}

extra_dir = "data/ZTF19aatubsj_mag/"
for name in sorted(os.listdir(extra_dir)):
    fid = name.split(".")[0]
    if fid in filtermap.keys():
        path = os.path.join(extra_dir, name)
        new = pd.read_table(path, sep="\s+", names=["date_met", "lim_mag", "mag", "mag_err"])
        
        
        
        mjd = [(swift_ref_time  + (x * u.s)).mjd for x in new["date_met"]]
        
#         print(new["date_met"][0]/(60*60*24*365))
        
        new = new.assign(filter=filtermap[fid], date_mjd=mjd)
        swift_data = pd.concat([new, swift_data])

print(swift_data)
# print(os.listdir(extra_dir))

In [ ]:
det_mask = np.logical_and(
    np.logical_and(
        photometry["magpsf"] != 99.,
#         photometry["instrument"] == "P48+ZTF"
        True
    ),
    photometry["isdiffpos"] == True
)

obs = photometry[det_mask]
lim = photometry[~det_mask]

plt.figure(figsize=(base_width, base_height), dpi=dpi)
ax = plt.subplot(111)
ax1b = ax.twinx()

cmap = {
    "g": "g",
    "r": "r",
    "i": "orange",
    "UVW2": "violet",
    "UVM2": "purple",
    "UVW1": "darkblue",
    "U": "lightblue",
#     "u": "lightblue",
#     "B": "blue",
#     "z": "darkorange",
#     "V": "grey"
}

# for f in list(set(obs["filter"])):

delta = None

for f in cmap.keys():
    
    if f in list(set(obs["filter"])):
    
        df = obs[obs["filter"] == f]

    #     factor = (df["isdiffpos"])*2 - 1.

    #     mag = -2.5 * np.log10(10**(-0.4*refmag[f[-1]]) + factor * 10.**(-0.4*df["magpsf"]))

    #     mask = factor > 0.

        ax.errorbar(df["jdobs"]-2400000.5, df["magpsf"], yerr=df["sigmamagpsf"], color=cmap[f], marker="o", linestyle=" ", label=f)

        delta = np.mean(df["magpsf"] - df["absmag"])
        print(delta)

        ax1b.errorbar(df["jdobs"]-2400000.5, df["absmag"], color=cmap[f], yerr=df["sigmamagpsf"], marker="o", linestyle=" ")

for f in filtermap.values():
    
    df = swift_data[swift_data["filter"] == f]
    
    ax.errorbar(df["date_mjd"], df["mag"], yerr=df["mag_err"], color=cmap[f], marker="o", linestyle=" ", label=f)
    ax1b.errorbar(df["date_mjd"], df["mag"] - delta, yerr=df["mag_err"], color=cmap[f], marker="o", linestyle=" ")
    #     ax.axhline(refmag[f[-1]], color=cmap[f[-1]], linestyle="--")
    
#     ldf = lim[lim["filter"] == f]    
#     limmag = -2.5 * np.log10(10**(-0.4*refmag[f[-1]]) + 10.**(-0.4*ldf["limmag"]))
#     ax.errorbar(ldf["jdobs"]-2400000.5, limmag, color=cmap[f[-1]], linestyle=" ", uplims=True)
#     ax1b.errorbar(ldf["jdobs"]-2400000.5, limmag - delta, color=cmap[f[-1]], linestyle=" ", uplims=True, marker="v")
    
ax.invert_yaxis()
ax1b.invert_yaxis()
ax.set_ylabel(r"Apparent Magnitude", fontsize=big_fontsize)
ax1b.set_ylabel(rf"Absolute Magnitude (z={redshift-1.:.3f})", fontsize=big_fontsize)
ax.tick_params(axis='both', which='major', labelsize=big_fontsize)
ax1b.tick_params(axis='both', which='major', labelsize=big_fontsize)
ax.set_xlabel("Date (MJD)", fontsize=big_fontsize)

t_neutrino = Time("2020-05-30T07:54:29.43", format='isot', scale='utc')


ax.axvline(t_neutrino.mjd, linestyle=":", label="IC200530A")

ax.legend(fontsize=big_fontsize, ncol=2)

filename = "AT2019fdr_lightcurve.pdf"

output_path = os.path.join(output_folder, f"ZTF/{filename}")
plt.ylim()

plt.savefig(f"plots/{filename}")
plt.savefig(output_path)


# plt.yscale("log")

In [ ]:
contour = pd.read_table("data/run00134139.evt000035473338.HESE.contour_90.txt", sep=" ", names=["ra", "dec"], comment='"')

In [ ]:
print(contour)

In [ ]:
plt.figure(figsize=(base_width, base_height), dpi=dpi)
plt.plot(np.degrees(contour["ra"]), np.degrees(contour["dec"]))
plt.scatter(257.2785777, 26.8557286, color="k", marker="*", s=100)

plt.ylabel("Declination [deg]", fontsize=big_fontsize)
plt.tick_params(axis='both', which='major', labelsize=big_fontsize)
plt.tick_params(axis='both', which='major', labelsize=big_fontsize)
plt.xlabel("Right Ascension [deg]", fontsize=big_fontsize)
plt.gca().invert_xaxis()

plt.grid()

filename = "tywin_position.pdf"

output_path = os.path.join(output_folder, f"ZTF/{filename}")

plt.savefig(f"plots/{filename}")
plt.savefig(output_path)

In [ ]:
t_disc = Time("2019-05-03T00:00:00", format='isot', scale='utc')

spectra_paths = [
    "2019-06-16.16_HET_HET-LRS_TexSNs.dat"
]

plt.figure(figsize=(base_width, base_height), dpi=dpi)
ax1 = plt.subplot(111)
j = 0

# Redo with actual numbers, not Robert-on-train-reads-off-plot

cols = ["C9", "C7", "k", "k"]
# cols = [":", "--", "-.", "-"]

lines = [
    (r"$\rm{H\alpha}$", 6562.8, 0),
    (r"$\rm{H\beta}$", 4861, 0),
    (r"$\rm{H\gamma}$", 4340, 0),
#     (r"$\rm{He\Romanbar{II}}$", 4686, 1),
#     (r"$\rm{N\Romanbar{III}}$", 4640, 2),
#     (r"$\rm{N\Romanbar{III}}$", 4100, 2),
#     (r"$\rm{O\Romanbar{III}}$", 3760, 3)
]

for base_path in spectra_paths:
    path = os.path.join("data", base_path)
    
    name = os.path.basename(path).split("_")
    vetos = ["P60"]
    
    raw_date = name[0]
    date = Time(f"{raw_date[:10]}T00:00:00.00", format='isot', scale='utc')
    n_days = date.mjd - t_disc.mjd
    
    if np.sum([x in name for x in vetos]) == 0:
        
        if "ascii" in path:
            data = pd.read_table(path, names=["wl", "flux", "flux_err"], sep=" ", comment='#')
        else:
            data = pd.read_csv(path, names=["wl", "flux"], sep="  ", index_col=None)
            print(data)
        mask = data["flux"] > 0.
        data[~mask] = np.nan
        
        if np.sum(~mask) == 1.:
            index = list(mask).index(0)
            lower = data[:index]
            upper = data[index:]
#             lower["flux"] *= lower["wl"]**2/np.mean(lower["wl"]**2)
            lower["flux"] /= np.mean(lower["flux"])
            upper["flux"] /= np.mean(upper["flux"])

            lower["flux"] *= upper["flux"].iloc[1]/lower["flux"].iloc[-1]
        
        label = f"+{n_days:.0f} days ({name[2][:4]})"
        y_pos = 0.5 - float(j)
        
        plt.plot(data["wl"]/redshift, y_pos + data["flux"]/np.mean(data["flux"]), label=label)
        plt.annotate(label, (5000., y_pos + 0.5), color=f"C{int(j/2)}", fontsize=big_fontsize)
        j += 2.0
# plt.yscale("log")
# plt.legend(fontsize=big_fontsize)

for (label, wl, col) in lines:
    plt.axvline(wl, linestyle=":", color=cols[col])
    
    bbox = dict(boxstyle="round", fc="white", ec=cols[col])
    
    plt.annotate(label, (wl + 40., 3.5 + 0.9*col), fontsize=big_fontsize, bbox=bbox, color=cols[col])
    
bbox = dict(boxstyle="circle", fc="white", ec="k")

# plt.annotate("+", (7150., 16.6), fontsize=big_fontsize, bbox=bbox, color=cols[col])

plt.ylabel(r"$F_{\lambda}$ + offset", fontsize=big_fontsize)
ax1.set_ylim(top=5.5)
ax1b = ax1.twiny()
ax1.set_xlim(left=3500, right=8000)
rslim = ax1.get_xlim()
ax1b.set_xlim((rslim[0] * redshift, rslim[1] * redshift))
ax1.set_xlabel(r"Rest Wavelength ($\rm \AA$)", fontsize=big_fontsize)
ax1b.set_xlabel(rf"Observed Wavelength (z={redshift-1.:.3f})", fontsize=big_fontsize)
ax1.tick_params(axis='both', which='major', labelsize=big_fontsize)
ax1b.tick_params(axis='both', which='major', labelsize=big_fontsize)
plt.tight_layout()

filename = "tywin_spectra.pdf"

output_path = os.path.join(output_folder, f"ZTF/{filename}")

plt.savefig(f"plots/{filename}")
plt.savefig(output_path)

In [ ]:
area_ztf = 1.37 + 21.57 + 4.52 + 4.09 + 20.56 + 6.22 + 20.06 + 2.66 + (0.9 * 25.3)
print(f"Summed ZTF area is {area_ztf:.2f} sq.deg.")
ztf_n_tde = 5.

# ZTF can observe between -30 deg and +90 deg


# frac_ztf = 0.5

# Take half?

ztf_sky = 28000.

n_year_ztf = 2.0
name = "NLSy1"

print(f"ZTF can access {ztf_sky:.2f} sq. deg of sky over a year.")

ztf_tde_disc_rate = ztf_n_tde / n_year_ztf / ztf_sky

print(f"ZTF found {ztf_n_tde} {name} in {n_year_ztf} years in 33000 sq. deg, "
      f"giving {ztf_tde_disc_rate:.2g} new {name}s per sq. deg per year or "
      f"giving {(ztf_tde_disc_rate/365.25):.2g} new {name}s per sq. deg per year")

n_year = 1.0

ztf_tde_rate = ztf_tde_disc_rate * n_year

print(f"We assume that each {name} is visible for an average of {n_year} years, giving a detection rate of {ztf_tde_rate:.2g} observed {name}s per sq deg at any one time.")

ztf_expectation = ztf_tde_rate * area_ztf
print(f"Given a {name} rate of {ztf_tde_rate:.3g} per sq. deg in ZTF, we have a final rate of {ztf_tde_rate:.2g}")
print(f"We thus expect {ztf_expectation:.3g} across all observed neutrinos")